In [ ]:
import timeit
import numpy as np
import pandas as pd
import math
import sys
import logging
import matplotlib.pyplot as plt
from IPython.display import display, HTML  # safe table display without .style

from speckit.schedulers import vectorized_ltf_plan, ltf_plan

In [ ]:
test_scenarios = [
    {"description": "Small", "params": {'N': 100_000, 'Jdes': 500, 'fs': 1000, 'olap': 0.5, 'bmin': 1.5, 'Lmin': 32, 'Kdes': 50}},
    {"description": "Typical", "params": {'N': 1_000_000, 'Jdes': 2000, 'fs': 1000, 'olap': 0.5, 'bmin': 1.5, 'Lmin': 128, 'Kdes': 100}},
    {"description": "High-Res (Jdes)", "params": {'N': 1_000_000, 'Jdes': 10000, 'fs': 1000, 'olap': 0.5, 'bmin': 1.5, 'Lmin': 128, 'Kdes': 100}},
    {"description": "Long Data (N)", "params": {'N': 10_000_000, 'Jdes': 2000, 'fs': 1000, 'olap': 0.5, 'bmin': 1.5, 'Lmin': 128, 'Kdes': 100}},
]
num_runs = 10
results_data = []

print("Comparing Performance of LTF Schedulers (running...)")

vectorized_ltf_plan(**(test_scenarios[0]["params"]))

for scenario in test_scenarios:
    params = scenario["params"]
    
    # Use globals() in notebook environment for timeit
    iter_times = timeit.repeat(lambda: ltf_plan(**params), number=1, repeat=num_runs)
    vec_times = timeit.repeat(lambda: vectorized_ltf_plan(**params), number=1, repeat=num_runs)

    iter_avg_time = np.mean(iter_times)
    vec_avg_time = np.mean(vec_times)
    speedup = iter_avg_time / vec_avg_time if vec_avg_time > 0 else float('inf')
    
    results_data.append({
        "Scenario": scenario["description"],
        "Iterative Time (s)": iter_avg_time,
        "Vectorized Time (s)": vec_avg_time,
        "Speedup (x)": speedup
    })

In [ ]:
results_df = pd.DataFrame(results_data)

# Make a formatted copy for display
fmt_df = results_df.copy()
fmt_df["Iterative Time (s)"] = fmt_df["Iterative Time (s)"].map(lambda v: f"{v:.4f}")
fmt_df["Vectorized Time (s)"] = fmt_df["Vectorized Time (s)"].map(lambda v: f"{v:.4f}")
fmt_df["Speedup (x)"] = fmt_df["Speedup (x)"].map(lambda v: f"{v:.2f}x")

caption_html = "<h3>Performance Benchmark Results</h3>"
table_html = fmt_df.to_html(index=False, escape=False)  # pure pandas HTML, no jinja2
display(HTML(caption_html + table_html))

In [ ]:
labels = results_df['Scenario']
iter_times = results_df['Iterative Time (s)']
vec_times = results_df['Vectorized Time (s)']
speedups = results_df['Speedup (x)']
x = np.arange(len(labels))
width = 0.35

fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10), sharex=True)
fig.suptitle('LTF Scheduler Performance Comparison', fontsize=16)

# --- Plot 1: Execution Time ---
rects1 = ax1.bar(x - width/2, iter_times, width, label='Iterative (ltf_plan)', color='skyblue')
rects2 = ax1.bar(x + width/2, vec_times, width, label='Vectorized', color='coral')
ax1.set_ylabel('Execution Time (seconds)')
ax1.set_title('Execution Time per Scenario (Log Scale)')
ax1.set_yscale('log')
ax1.legend()
ax1.bar_label(rects1, padding=3, fmt='%.3f')
ax1.bar_label(rects2, padding=3, fmt='%.3f')
ax1.grid(axis='y', linestyle='--', alpha=0.7)

# --- Plot 2: Speedup Factor ---
rects3 = ax2.bar(x, speedups, color='mediumseagreen')
ax2.set_ylabel('Speedup Factor (x)')
ax2.set_title('Speedup Factor of Vectorized vs. Iterative')
ax2.set_xticks(x)
ax2.set_xticklabels(labels, rotation=0)
ax2.bar_label(rects3, padding=3, fmt='%.1fx')
ax2.axhline(1, color='grey', linestyle='--', lw=1) # Add a line at 1x for reference
ax2.grid(axis='y', linestyle='--', alpha=0.7)

fig.tight_layout(rect=[0, 0, 1, 0.96])
plt.show()